In [7]:
import sys
sys.path.append('..')

from spyral.core.constants import QBRHO_2_P
from spyral.core.run_stacks import form_run_string

from spyral_utils.nuclear import NuclearDataMap
from spyral_utils.nuclear.target import GasTarget, load_target
from spyral_utils.plot import Histogrammer

from pathlib import Path
from scipy.constants import physical_constants, torr
from scipy.integrate import quad
from scipy.stats import iqr
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import vector
import lmfit

%matplotlib widget

In [8]:
target_material_path = Path("C:\\Users\\schaeffe\\Desktop\\e20009-analysis\\e20009_parameters\\e20009_target.json")

In [9]:
# Run number range (inclusive)
run_min = 108
run_max = 320

# The nucleus we observe (the one fitted)
ejectile_z = 1
ejectile_a = 1

# The incoming nucleus (the beam)
projectile_z = 5
projectile_a = 10

# The target nucleus
target_z = 1
target_a = 2

residual_z = target_z + projectile_z - ejectile_z
residual_a = target_a + projectile_a - ejectile_a

if residual_z < 0:
    raise Exception(f"Illegal nuclei! Residual Z: {residual_z}")
if residual_a < 1:
    raise Exception(f"Illegal nuclei! Residual A: {residual_a}")

In [10]:
# Setup nuclear data objects
nuclear_map = NuclearDataMap()

target_material = load_target(target_material_path, nuclear_map)
if not isinstance(target_material, GasTarget):
    print('Target error!')

ejectile = nuclear_map.get_data(ejectile_z, ejectile_a)
projectile = nuclear_map.get_data(projectile_z, projectile_a)
target = nuclear_map.get_data(target_z, target_a)
residual = nuclear_map.get_data(residual_z, residual_a)

print(f"Reaction: {target}({projectile}, {ejectile}){residual}")
print(f"Target material: {target_material.ugly_string}")

Reaction: 2H(10B, 1H)11B
Target material: (Gas)2H2


In [11]:
results = {}

# Add keys for result parameters
results["polar"] = np.empty(0, float)
results["kinetic_energy"] = np.empty(0, float)
results["ex_energy"] = np.empty(0, float)
results["cm_polar"] = np.empty(0, float)
results["chisq"] = np.empty(0, float)
results["z_vert"] = np.empty(0, float)

In [12]:
proj_energy_start = 106 # Beam Energy, MeV

In [13]:
target_vector = vector.array({"px": [0.0], "py": [0.0], "pz": [0.0], "E": [target.mass]})

print(f"Target Vector: {target_vector}")


# Fitted proton vertices 297_590868 ##
# x, y, z
# proton_x_vertex = -0.01995104
# proton_y_vertex = -0.00139850
# proton_z_vertex = 0.39123998

# proton_fitted_brho = 0.71039552 
# proton_fitted_polar = 0.75429091 # radian
# proton_fitted_azimuthal = 4.73009530 # radian

# Fitted proton vertices 256_597527 ##

proton_x_vertex = 0.00503193
proton_y_vertex = 0.00584341
proton_z_vertex = 0.28346858
proton_fitted_brho = 0.41477857
proton_fitted_polar = 1.21602552 # radian
proton_fitted_azimuthal = 1.32251477 # radian


vertices = np.array([proton_x_vertex, proton_y_vertex, proton_z_vertex])
distance = np.linalg.norm(vertices)  # This is a float, no need to convert
distances = np.array([distance])  # Store as an array if needed

projectile_ke = proj_energy_start - target_material.get_energy_loss(projectile, proj_energy_start, distances)
# print(vertices)
# print(distances)  
# print(projectile_ke)

projectile_vector = vector.array({                                           # Amount of energy lost by beam
        "px": np.zeros(len(projectile_ke)),
        "py": np.zeros(len(projectile_ke)),
        "pz": np.sqrt(projectile_ke * (projectile_ke + 2.0 * projectile.mass)),
        "E": projectile_ke + projectile.mass
    })             

print(f"Projectile Vector: {projectile_vector}") 

momentum = proton_fitted_brho * float(ejectile.Z) * QBRHO_2_P
kinetic_energy = np.sqrt(momentum**2.0 + ejectile.mass**2.0) - ejectile.mass

ejectile_vector = vector.array({
        "px": momentum * np.sin(proton_fitted_polar) * np.cos(proton_fitted_azimuthal),
        "py": momentum * np.sin(proton_fitted_polar) * np.sin(proton_fitted_azimuthal),
        "pz": momentum * np.cos(proton_fitted_polar),
        "E": np.sqrt(momentum**2.0 + ejectile.mass**2.0)
})

print(f"Ejectile Vector: {ejectile_vector}") 

residual_vector = target_vector + projectile_vector - ejectile_vector # type: ignore
ex_energy = residual_vector.mass - residual.mass # Excitation energy is "extra" mass

print(f"B11 Excitation Energy: {ex_energy}")

Target Vector: [(0., 0., 0., 1875.6129314)]
Projectile Vector: [(0., 0., 1378.36938188, 9425.76368529)]
Ejectile Vector: (28.65407307, 113.0283625, 43.19526475, 946.47598271)
B11 Excitation Energy: [15.25038914]
